In [5]:
import sqlite3, random, names, time

In [6]:
def create_database():
    conn = sqlite3.connect('data.db')

    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS equipes (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            elo INT,
            titre_mondial TEXT,
            titre_continental TEXT,
            victoires INT,
            nuls INT,
            defaites INT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS matchs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            home_team_name TEXT NOT NULL,
            away_team_name TEXT NOT NULL,
            home_team_score INTEGER,
            away_team_score INTEGER
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS joueurs (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            nation TEXT NOT NULL,
            valeur INT,
            titre_mondial TEXT,
            titre_continental TEXT,
            buts INT, 
            tirs_cadres INT,
            tirs INT,
            passes_reusites INT,
            passes INT,
            interceptions INT,
            arrets INT,
            victoires INT,
            nuls INT,
            defaites INT
       )
    ''')

    conn.commit()
    conn.close()


In [7]:
def reset_database():
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute('DROP TABLE IF EXISTS equipes')
    cursor.execute('DROP TABLE IF EXISTS matchs')
    cursor.execute('DROP TABLE IF EXISTS joueurs')
    conn.commit()
    conn.close()


In [8]:
def update_equipes_database(name, elo, titre_mondial, titre_continental, victoires, nuls, defaites):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    cursor.execute('''
        UPDATE equipes 
        SET elo = ?,
            titre_mondial = ?,
            titre_continental = ?,
            victoires = ?,
            nuls = ?,
            defaites = ?
        WHERE name = ?;
    ''', (elo, titre_mondial, titre_continental, victoires, nuls, defaites, name))

    conn.commit()
    conn.close()

def update_joueur_database(name, valeur, buts, tirs_cadrés, tirs, passes_reusites, passes, interceptions, arrets, titre_mondial, titre_continental, victoires, nuls, defaites):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    cursor.execute('''
        UPDATE joueurs 
        SET valeur = ?, 
            buts = ?, 
            tirs_cadres = ?,
            tirs = ?,
            passes_reusites = ?,
            passes = ?,
            interceptions = ?,
            arrets = ?,
            titre_mondial = ?,
            titre_continental = ?,
            victoires = ?,
            nuls = ?,
            defaites = ?
        WHERE name = ?;
    ''', (valeur, buts, tirs_cadrés, tirs, passes_reusites, passes, interceptions, arrets, titre_mondial, titre_continental, victoires, nuls, defaites, name))

    conn.commit()
    conn.close()


In [9]:
def add_equipe(name, elo):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO equipes (name, elo, titre_mondial, titre_continental, victoires, nuls, defaites) VALUES (?, ?, ?, ?, ?, ?, ?)', (name, elo, "", "", 0, 0, 0))
    conn.commit()
    conn.close()

def add_match(home_team_name, away_team_name, home_team_score, away_team_score):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO matchs (home_team_name, away_team_name, home_team_score, away_team_score)
        VALUES (?, ?, ?, ?)
    ''', (home_team_name, away_team_name, home_team_score, away_team_score))
    conn.commit()
    conn.close()

def add_joueur(name, nation):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO joueurs (name, nation, valeur, titre_mondial, titre_continental, buts, tirs_cadres, tirs, passes_reusites, passes, interceptions, arrets, victoires, nuls, defaites)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (name, nation, 0, "", "", 0, 0, 0, 0, 0, 0, 0, 0, 0, 0))
    conn.commit()
    conn.close()


In [10]:
class Joueur:
    def __init__(self, nom, stat, num):
        self.nom: str = nom
        self.stat: dict = stat
        self.num: int = num
        self.statistiques = {'buts': 0, 'tirs cadrés': 0, 'tirs': 0, 'passes reusites': 0, 'passes': 0, 'interceptions': 0,  'arrets': 0, 'victoires': 0, 'nuls': 0, 'defaites': 0}
        self.palmarès = {"mondiale": "", "continentale": ""}
        self.valeur = 10000 * (self.statistiques['buts'] * 10 + self.statistiques['arrets'] * 9 + self.statistiques['tirs cadrés'] * 8 + self.statistiques['interceptions'] * 4 + self.statistiques['tirs'] * 3 + self.statistiques['passes reusites'] * 2 + self.statistiques['passes'])

    def passe(self):
        chance = random.uniform(0, 100)
        self.statistiques['passes'] += 1
        if chance < self.stat["passe"]:
            # print(f"{self.nom} fait la passe.")
            self.statistiques['passes reusites'] += 1
            return True
        else:
            # print(f"{self.nom} rate sa passe !")
            return False

    def defense(self, other, gardien=False):
        chance = random.uniform(0, 100)
        if chance < self.stat["defense"] and not gardien:
            self.statistiques['interceptions'] += 1
            # print(f"Mais {self.nom} intecepte la passe de {other.nom} !")
            return True
        elif chance < self.stat["defense"] and gardien:
            self.statistiques['arrets'] += 1
            # print(f"Mais {self.nom} arrete le ballon !")
            return True
        elif chance >= self.stat["defense"] and gardien:
            # print(f"But de {other.nom} !!!")
            pass
        return False

    def tir(self):
        chance = random.uniform(0, 100)
        self.statistiques['tirs'] += 1
        if chance < self.stat["tir"]:
            self.statistiques['tirs cadrés'] += 1
            # print(f"{self.nom} cadre son tir !")
            return True
        else:
            # print(f"{self.nom} manque son tir !")
            return False
        
    def update(self):
        self.valeur = 10000 * (self.statistiques['buts'] * 10 + self.statistiques['arrets'] * 9 + self.statistiques['tirs cadrés'] * 8 + 
                               self.statistiques['interceptions'] * 4 + self.statistiques['tirs'] * 3 + self.statistiques['passes reusites'] * 2 + 
                               self.statistiques['passes'])


In [11]:
class Equipe:
    def __init__(self, nom):
        self.nom: str = nom
        self.joueurs = self.creer_equipe()
        self.elo = 1000
        self.palmarès = {"mondiale": "", "continentale": ""}
        self.statistiques = {"victoires": 0, "nuls": 0, "defaites": 0}

    def creer_equipe(self):
        return {"gardien": [Joueur(names.get_full_name(gender='male'),
                                   {"passe": random.uniform(75, 100), "tir": random.uniform(25, 100),
                                    "defense": random.uniform(75, 100)}, 1)],
                "defenseurs": [Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(50, 100),
                                       "defense": random.uniform(50, 100)}, 2),
                               Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(50, 100),
                                       "defense": random.uniform(50, 100)}, 3),
                               Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(50, 100),
                                       "defense": random.uniform(50, 100)}, 4),
                               Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(50, 100),
                                       "defense": random.uniform(50, 100)}, 5)],
                "millieux": [Joueur(names.get_full_name(gender='male'),
                                    {"passe": random.uniform(75, 100), "tir": random.uniform(50, 100),
                                     "defense": random.uniform(25, 100)}, 6),
                             Joueur(names.get_full_name(gender='male'),
                                    {"passe": random.uniform(75, 100), "tir": random.uniform(50, 100),
                                     "defense": random.uniform(25, 100)}, 7),
                             Joueur(names.get_full_name(gender='male'),
                                    {"passe": random.uniform(75, 100), "tir": random.uniform(50, 100),
                                     "defense": random.uniform(25, 100)}, 8)],
                "attaquants": [Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(75, 100),
                                       "defense": random.uniform(25, 100)}, 9),
                               Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(75, 100),
                                       "defense": random.uniform(25, 100)}, 10),
                               Joueur(names.get_full_name(gender='male'),
                                      {"passe": random.uniform(50, 100), "tir": random.uniform(75, 100),
                                       "defense": random.uniform(25, 100)}, 11)]
                }

    def set_equipe(self):
        self.joueurs = self.creer_equipe()
        catos = ['gardien', 'defenseurs', 'millieux', 'attaquants']
        for cato in catos:
            for joueur in self.joueurs[cato]:
                add_joueur(joueur.nom, self.nom)

    def get_elo(self):
        return self.elo

    def set_elo(self, value):
        self.elo = value
        
    def update_resultat(self, typo):
        self.statistiques[typo] += 1
        catos = ['gardien', 'defenseurs', 'millieux', 'attaquants']
        for cato in catos:
            for joueur in self.joueurs[cato]:
                joueur.statistiques[typo] += 1

    def add_titre(self, typo, value):
        self.palmarès[typo] += value+"\n"
        catos = ['gardien', 'defenseurs', 'millieux', 'attaquants']
        for cato in catos:
            for joueur in self.joueurs[cato]:
                joueur.palmarès[typo] += value+"\n"
    
    def update_joueurs(self):
        catos = ['gardien', 'defenseurs', 'millieux', 'attaquants']
        for cato in catos:
            for joueur in self.joueurs[cato]:
                joueur.update()
                update_joueur_database(joueur.nom, joueur.valeur, joueur.statistiques['buts'], joueur.statistiques['tirs cadrés'], 
                                       joueur.statistiques['tirs'], joueur.statistiques['passes reusites'], joueur.statistiques['passes'], 
                                       joueur.statistiques['interceptions'], joueur.statistiques['arrets'], joueur.palmarès["mondiale"], 
                                       joueur.palmarès['continentale'], joueur.statistiques['victoires'], joueur.statistiques['nuls'], 
                                       joueur.statistiques['defaites'])


In [12]:
def match(e1, e2):
    stat_match = {e1.nom: {"but": 0, "tir cadré": 0, "tir": 0, "passe réussite": 0, "passe raté": 0, "interception": 0,
                           "arret": 0},
                  e2.nom: {"but": 0, "tir cadré": 0, "tir": 0, "passe réussite": 0, "passe raté": 0, "interception": 0,
                           "arret": 0}}

    pile_face = random.randint(0, 1)
    for i in range(2):
        #print(f"Début de la {1 + i} Mi-temps {e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")
        position = 3
        if i == 0:
            if pile_face == 0:
                possession, adversaire = e1, e2
            elif pile_face == 1:
                possession, adversaire = e2, e1
            balle = random.choice(possession.joueurs["millieux"])
        elif i == 1:
            if pile_face == 1:
                possession, adversaire = e1, e2
            elif pile_face == 0:
                possession, adversaire = e2, e1
            balle = random.choice(possession.joueurs["millieux"])
        for j in range(90):
            # Dans la surface adverse - occassion de but
            if (position == 5 and possession == e1) or (position == 1 and possession == e2):
                action = balle.tir()
                if action:
                    action = adversaire.joueurs["gardien"][0].defense(balle, gardien=True)
                    if not action:
                        balle.statistiques['buts'] += 1
                        stat_match[possession.nom]["but"] += 1
                        position = 3
                        balle = random.choice(adversaire.joueurs["millieux"])
                        #print(f"{e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")

                    else:
                        balle = random.choice(adversaire.joueurs["defenseurs"] + adversaire.joueurs["gardien"])
                        possession, adversaire = adversaire, possession
                else:
                    perte = random.randint(0, 1)
                    if perte == 0:
                        balle = random.choice(adversaire.joueurs["defenseurs"])
                        possession, adversaire = adversaire, possession
                    else:
                        balle = random.choice(possession.joueurs["attaquants"])


            else:
                action = balle.passe()
                if action:

                    # Dans sa surface - dégagement
                    if (position == 1 and possession == e1) or (position == 5 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["attaquants"])
                        receveur = random.choice(possession.joueurs["defenseurs"])

                    # Dans son terrain
                    elif (position == 2 and possession == e1) or (position == 4 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["attaquants"] + adversaire.joueurs["millieux"])
                        receveur = random.choice(possession.joueurs["millieux"] + possession.joueurs["defenseurs"])

                    # Dans le millieu du terrain
                    elif (position == 3 and possession == e1) or (position == 3 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["millieux"] + adversaire.joueurs["defenseurs"])
                        receveur = random.choice(possession.joueurs["attaquants"] + possession.joueurs["millieux"])

                    # Dans le terrain d'adverse - action dangereuse
                    elif (position == 4 and possession == e1) or (position == 2 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["defenseurs"])
                        receveur = random.choice(possession.joueurs["attaquants"])
                    action = defenseur.defense(balle)
                    if action:
                        possession, adversaire = adversaire, possession
                        balle = defenseur

                    else:
                        balle = receveur
                        if possession == e1:
                            position += 1
                        elif possession == e2:
                            position -= 1

                else:
                    perte = random.randint(0, 1)
                    if perte == 0:
                        balle = random.choice(adversaire.joueurs["millieux"])
                        possession, adversaire = adversaire, possession
                    else:
                        balle = random.choice(possession.joueurs["millieux"])
    #print(f"Fin du temps reglementaire {e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")
    print(f"{e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")
    return stat_match


In [13]:
def generate_ligue(liste):
    matchs = []
    if len(liste) % 2 == 1: liste.append(None)
    for j in range(len(liste) - 1):
        for i in range(len(liste) // 2):
            matchs.append((liste[i], liste[len(liste) - 1 - i]))
        liste.insert(1, liste.pop())
    matchs = [match for match in matchs if not None in match]
    if None in liste: liste.remove(None)
    return matchs


In [14]:
def valeur(elo):
    if elo < 1000: k = 80
    if elo >= 1000 and elo < 2000: k = 50
    if elo >= 2000 and elo <= 2400: k = 30
    if elo > 2400: k = 20
    return k

def estimation(e1, e2):
    esp = (e1 - e2) / 400
    return 1 / (1 + 10 ** esp)

def calcul_elo(player, e2, resultat):
    if resultat == 1: 
        player.update_resultat("victoires")
    elif resultat == 0.5: 
        player.update_resultat("nuls")
    elif resultat == 0: 
        player.update_resultat("defaites")
    e1 = player.get_elo()
    k = valeur(e1)
    estime = estimation(e1, e2)
    rang = e1 + k * (resultat - estime)
    if rang < 1: rang = 1
    return player.set_elo(round(rang, 3))


In [15]:
def match_ligue(equipes):
    ligue = generate_ligue(equipes)
    classement = {equipe: {"points": 0, "buts marqués": 0, "buts encaissés": 0} for equipe in equipes}
    current = classement
    for matchs in ligue:
        data = match(matchs[0], matchs[1])
        add_match(matchs[0].nom, matchs[1].nom, data[matchs[0].nom]["but"], data[matchs[1].nom]["but"])
        classement[matchs[0]]["buts marqués"] += data[matchs[0].nom]["but"]
        classement[matchs[1]]["buts marqués"] += data[matchs[1].nom]["but"]
        classement[matchs[0]]["buts encaissés"] += data[matchs[1].nom]["but"]
        classement[matchs[1]]["buts encaissés"] += data[matchs[0].nom]["but"]
        if data[matchs[0].nom]["but"] == data[matchs[1].nom]["but"]:
            classement[matchs[0]]["points"] += 1
            classement[matchs[1]]["points"] += 1
            calcul_elo(matchs[1], matchs[0].get_elo(), 0.5)
            calcul_elo(matchs[0], matchs[1].get_elo(), 0.5)
        elif data[matchs[0].nom]["but"] < data[matchs[1].nom]["but"]:
            classement[matchs[1]]["points"] += 3
            calcul_elo(matchs[1], matchs[0].get_elo(), 1)
            calcul_elo(matchs[0], matchs[1].get_elo(), 0)
        elif data[matchs[0].nom]["but"] > data[matchs[1].nom]["but"]:
            classement[matchs[0]]["points"] += 3
            calcul_elo(matchs[1], matchs[0].get_elo(), 0)
            calcul_elo(matchs[0], matchs[1].get_elo(), 1)
        current = sorted(sorted(sorted(classement.items(), key=lambda x: x[1]['buts marqués'], reverse=True),
                                key=lambda x: x[1]['buts marqués'] - x[1]['buts encaissés'], reverse=True),
                         key=lambda x: x[1]['points'], reverse=True)
    return current


In [16]:
def creer_equipes(liste):
    equipes = []
    for nom in liste: equipes.append(Equipe(nom))
    return equipes


def creer_groupe(n, liste):
    lst = [[] for i in range(n)]
    for i in range(len(liste)):
        lst[i % n].append(liste[i])
    return lst


In [17]:
def qualifiés(n, classement):
    for equipe in classement: print(
        f"{equipe[0].nom} ({equipe[0].elo}) | Points : {equipe[1]['points']} | Buts Marqués : {equipe[1]['buts marqués']} | Buts Encaissés : {equipe[1]['buts encaissés']} | Différences : {equipe[1]['buts marqués'] - equipe[1]['buts encaissés']}")
    participants = {'winners': [classement[i][0] for i in range(n)],
                    'runners_ups': [classement[j + n][0] for j in range(n)]}
    return participants


def matchs_groupes(n, groupes):
    winners, runners_ups = [], []
    for groupe in groupes:
        classement_ligue = match_ligue(groupe)
        liste = qualifiés(n, classement_ligue)
        winners += liste['winners']
        runners_ups += liste['runners_ups']
    return winners, runners_ups


In [18]:
def tir_au_but(e1, e2):
    # Séance de tirs au but
    equipe1 = e1.joueurs["attaquants"] + e1.joueurs["millieux"] + e1.joueurs["defenseurs"] + e1.joueurs["gardien"]
    equipe2 = e2.joueurs["attaquants"] + e2.joueurs["millieux"] + e2.joueurs["defenseurs"] + e2.joueurs["gardien"]
    score1, score2 = 0, 0
    #print(f"Début de la scéance de tirs au buts.")

    for i in range(5):
        t1 = equipe1[i].stat["tir"] + random.uniform(0, 100)
        g2 = e2.joueurs["gardien"][0].stat["defense"] + random.uniform(0, 100)
        if t1 > g2:
            #print(f"Le tir de {equipe1[i].nom} est rentré !!")
            score1 += 1
        """else:
            print(f"{equipe1[i].nom} a raté son tir !!")"""

        t2 = equipe2[i].stat["tir"] + random.uniform(0, 100)
        g1 = e1.joueurs["gardien"][0].stat["defense"] + random.uniform(0, 100)
        if t2 > g1:
            #print(f"Le tir de {equipe2[i].nom} est rentré !!")
            score2 += 1
        """else:
            print(f"{equipe2[i].nom} a raté son tir !!")"""
    n = 5
    while True:
        if score1 != score2:
            #print(f"Fin de la séance tirs au buts : {e1.nom} {score1} - {score2} {e2.nom}")
            break
        if n == 11: n = 0
        t1 = equipe1[n].stat["tir"] + random.uniform(0, 100)
        g2 = e2.joueurs["gardien"][0].stat["defense"] + random.uniform(0, 100)
        if t1 > g2:
            #print(f"Le tir de {equipe1[n].nom} est rentré !!")
            score1 += 1
        """else:
            print(f"{equipe1[n].nom} a raté son tir !!")"""

        t2 = equipe2[n].stat["tir"] + random.uniform(0, 100)
        g1 = e1.joueurs["gardien"][0].stat["defense"] + random.uniform(0, 100)
        if t2 > g1:
            #print(f"Le tir de {equipe2[n].nom} est rentré !!")
            score2 += 1
        """else:
            print(f"{equipe2[n].nom} a raté son tir !!")"""
        n += 1
    if score1 > score2:
        return e1, e2
    elif score1 < score2:
        return e2, e1


In [19]:
def prolongation(e1, e2, score):
    stat_match = {
        e1.nom: {"but": score, "tir cadré": 0, "tir": 0, "passe réussite": 0, "passe raté": 0, "interception": 0,
                 "arret": 0},
        e2.nom: {"but": score, "tir cadré": 0, "tir": 0, "passe réussite": 0, "passe raté": 0, "interception": 0,
                 "arret": 0}}

    pile_face = random.randint(0, 1)

    for i in range(2):
        #print(f"Début de la {i + 1} Mi-temps des prolongations {e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")
        position = 3
        if i == 0:
            if pile_face == 0:
                possession, adversaire = e1, e2
            elif pile_face == 1:
                possession, adversaire = e2, e1
            balle = random.choice(possession.joueurs["millieux"])
        elif i == 1:
            if pile_face == 1:
                possession, adversaire = e1, e2
            elif pile_face == 0:
                possession, adversaire = e2, e1
            balle = random.choice(possession.joueurs["millieux"])
        for j in range(60):
            # Dans la surface adverse - occassion de but
            if (position == 5 and possession == e1) or (position == 1 and possession == e2):
                action = balle.tir()
                if action:
                    action = adversaire.joueurs["gardien"][0].defense(balle, gardien=True)
                    if not action:
                        balle.statistiques['buts'] += 1
                        stat_match[possession.nom]["but"] += 1
                        position = 3
                        balle = random.choice(adversaire.joueurs["millieux"])
                        #print(f"{e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")

                    else:
                        balle = random.choice(adversaire.joueurs["defenseurs"] + adversaire.joueurs["gardien"])
                        possession, adversaire = adversaire, possession
                else:
                    perte = random.randint(0, 1)
                    if perte == 0:
                        balle = random.choice(adversaire.joueurs["defenseurs"])
                        possession, adversaire = adversaire, possession
                    else:
                        balle = random.choice(possession.joueurs["attaquants"])

            else:
                action = balle.passe()
                if action:

                    # Dans sa surface - dégagement
                    if (position == 1 and possession == e1) or (position == 5 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["attaquants"])
                        receveur = random.choice(possession.joueurs["defenseurs"])

                    # Dans son terrain
                    elif (position == 2 and possession == e1) or (position == 4 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["attaquants"] + adversaire.joueurs["millieux"])
                        receveur = random.choice(possession.joueurs["millieux"] + possession.joueurs["defenseurs"])

                    # Dans le millieu du terrain
                    elif (position == 3 and possession == e1) or (position == 3 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["millieux"] + adversaire.joueurs["defenseurs"])
                        receveur = random.choice(possession.joueurs["attaquants"] + possession.joueurs["millieux"])

                    # Dans le terrain d'adverse - action dangereuse
                    elif (position == 4 and possession == e1) or (position == 2 and possession == e2):
                        defenseur = random.choice(adversaire.joueurs["defenseurs"])
                        receveur = random.choice(possession.joueurs["attaquants"])
                    action = defenseur.defense(balle)
                    if action:
                        possession, adversaire = adversaire, possession
                        balle = defenseur

                    else:
                        balle = receveur
                        if possession == e1:
                            position += 1
                        elif possession == e2:
                            position -= 1

                else:
                    perte = random.randint(0, 1)
                    if perte == 0:
                        balle = random.choice(adversaire.joueurs["millieux"])
                        possession, adversaire = adversaire, possession
                    else:
                        balle = random.choice(possession.joueurs["millieux"])

    #print(f"Fin des prolongations {e1.nom} {stat_match[e1.nom]['but']} - {stat_match[e2.nom]['but']} {e2.nom}")

    if stat_match[e1.nom]['but'] == stat_match[e2.nom]['but']:
        return tir_au_but(e1, e2)
    elif stat_match[e1.nom]['but'] > stat_match[e2.nom]['but']:
        return e1, e2
    elif stat_match[e1.nom]['but'] < stat_match[e2.nom]['but']:
        return e2, e1


In [20]:
def match_éliminatoire(e1, e2):
    data = match(e1, e2)
    add_match(e1.nom, e2.nom, data[e1.nom]["but"], data[e2.nom]["but"])
    if data[e1.nom]["but"] == data[e2.nom]["but"]:
        calcul_elo(e2, e1.get_elo(), 0.5)
        calcul_elo(e1, e2.get_elo(), 0.5)
        w, l = prolongation(e1, e2, data[e1.nom]["but"])
    elif data[e1.nom]["but"] < data[e2.nom]["but"]:
        calcul_elo(e2, e1.get_elo(), 1)
        calcul_elo(e1, e2.get_elo(), 0)
        w, l = e2, e1
    elif data[e1.nom]["but"] > data[e2.nom]["but"]:
        calcul_elo(e2, e1.get_elo(), 0)
        calcul_elo(e1, e2.get_elo(), 1)
        w, l = e1, e2
    return w, l


In [21]:
def séries_éliminatoire(liste, premier_tour=True):
    matchs = []
    trois_place = []
    if premier_tour:
        for i in range(len(liste[0])):
            winner, loser = match_éliminatoire(liste[0][i], liste[1][len(liste[1]) - (i + 1)])
            matchs.append(winner)
            trois_place.append(loser)
            final_four = len(liste[0]) + len(liste[1])

    else:
        for i in range(len(liste) // 2):
            winner, loser = match_éliminatoire(liste[i * 2], liste[i * 2 + 1])
            matchs.append(winner)
            trois_place.append(loser)
            final_four = len(liste)

    if final_four == 4:
        trois, quatre = match_éliminatoire(trois_place[0], trois_place[1])
        #print(f"Bravo a {trois.nom} qui gagne la troisième place.")
        #print(f"Finale entre {matchs[0].nom} et {matchs[1].nom}")
        champion, vice = match_éliminatoire(matchs[0], matchs[1])
        #print(f"{champion.nom} est sacré champion ! Félicitations !!!")
        print(f"3 eme place : {trois.nom} \nVice Champion : {vice.nom} \nChampion : {champion.nom}")
        return champion
    return séries_éliminatoire(matchs, premier_tour=False)


In [22]:
def competition(liste, pq, pg, nq, typo, titre):
    random.shuffle(liste)
    if pq == 0:
        equipes_qualifiées = liste
    else:
        qualifiquation = creer_groupe(pq, liste)
        chapeau1, chapeau2 = matchs_groupes(nq, qualifiquation)
        equipes_qualifiées = chapeau1 + chapeau2
    groupes = creer_groupe(pg, equipes_qualifiées)
    eliminatoire = matchs_groupes(nq, groupes)
    champion = séries_éliminatoire(eliminatoire)
    champion.add_titre(typo, f"{titre}")
    return champion


In [23]:
if __name__ == '__main__':
    start = time.time()
    reset_database()
    create_database()
    oceanie = ["Nouvelle-Zélande", "Salomon", "Nouvelle-Calédonie", "Papouasie-Nouvelle-Guinée", "Fidji", "Tahiti",
            "Vanuatu", "Samoa américaines", "Îles Cook", "Samoa", "Tonga"]
    nord = ["Mexique", "États-Unis", "Costa rica", "Canada", "Panama", "Jamaïque", "Salvador", "Honduras", "Curaçao",
            "Haïti", "Trinité-et-Tobago", "Guatemala", "Antigua-et-Barbuda",
            "Suriname", "Saint-Kitts-et-Nevis", "Nicaragua", "République dominicaine", "Barbade", "Cuba", "Bermudes",
            "Porto Rico", "Guyana", "Saint-Lucie", "Grenade", "Belize",
            "Saint-Vincent-et-Les-Grenadines", "Montserrat", "Dominique", "Îles Caïmans", "Aruba", "Bahamas",
            "Turks-et-Caicos", "Îles Vierges américaines", "Îles Vierges britanniques",
            "Anguilla"]
    sud = ["Brésil", "Argentine", "Uruguay", "Colombie", "Pérou", "Chili", "Équateur", "Paraguay", "Venezuela", "Bolivie"]
    afrique = ["Sénégal", "Maroc", "Tunisie", "Nigéria", "Cameroun", "Égypte", "Algérie", "Mali", "Côte d'Ivoire",
            "Burkina Faso", "Ghana", "Afrique du Sud", "République démocratique du Congo",
            "Cap-Vert", "Gabon", "Guinée", "Zambie", "Ouganda", "Bénin", "Guinée Équatoriale", "Congo", "Kenya",
            "Madagascar", "Mauritanie", "Mozambie", "Nambie", "Guinée-Bissau", "Sierra Leone",
            "Angola", "Libye", "Niger", "Malawi", "Zimbabwe", "Gambie", "Togo", "Soudan", "Comores", "Tanzanie",
            "République centrafricaine", "Rwanda", "Éthiopie", "Burundi", "Eswatini", "Lesotho",
            "Botswana", "Liberia", "Soudan du Sud", "Maurice", "Tchad", "São Tomé-et-Príncipe", "Djibouti", "Seychelles",
            "Érythrée", "Somalie"]
    asie = ["Iran", "Japon", "Corée du Sud", "Australie", "Qatar", "Arabie Saoudite", "Émirats arabes unis", "Irak", "Oman",
            "Ouzbékistan", "Chine", "Bahreïn", "Jordanie", "Syrie", "Palestine",
            "République Kirghize", "Vietnam", "Liban", "Inde", "Tadjikistan", "Thaïlande", "RDP Corée", "Philippines",
            "Turkménistan", "Hong Kong", "Malasie", "Koweït", "Indonésie", "Maldives",
            "Yémen", "Afghanistan", "Chinese Taipei", "Myanmar", "Singapour", "Népal", "Cambodge", "Macao", "Mongolie",
            "Bhoutan", "Laos", "Brunei", "Bangladesh", "Timor oriental", "Guam", "Sri Lanka", "Pakistan"]
    europe = ["Belgique", "France", "Angleterre", "Espagne", "Italie", "Pays-Bas", "Portugal", "Danmark", "Allemagne",
            "Croatie", "Suisse", "Pays de galles", "Suède", "Serbie", "Pologne", "Ukraine",
            "République tchèque", "Autriche", "Russie", "Norvège", "Hongrie", "Turquie", "Écosse", "République d'Irlande",
            "Grèce", "Slovaquie", "Roumanie", "Bosnie-et-Herzégovine", "Irlande du Nord",
            "Finlande", "Islande", "Macédoine du Nord", "Slovénie", "Albanie", "Monténégro", "Bulgarie", "Israël",
            "Géorgie", "Luxembourg", "Arménie", "Belarus", "Kosovo", "Estonie", "Chypre", "Kazakhstan",
            "Îles Féroé", "Azerbaïdjan", "Lettonie", "Lithuanie", "Andorre", "Malte", "Moldavie", "Liechtenstein",
            "Gibraltar", "Saint-Marin"]

    equipes_europe = creer_equipes(europe)
    equipes_sud = creer_equipes(sud)
    equipes_nord = creer_equipes(nord)
    equipes_afrique = creer_equipes(afrique)
    equipes_asie = creer_equipes(asie)
    equipes_oceanie = creer_equipes(oceanie)
    equipes = equipes_europe + equipes_sud + equipes_nord + equipes_afrique + equipes_asie + equipes_oceanie
    for equipe in equipes: add_equipe(equipe.nom, equipe.elo)
    debut = 2027
    for i in range(14):
        if i % 4 == 0:
            competition(equipes_afrique, 16, 8, 1, "continentale", f"champion d'afrique {debut + i}")
            competition(equipes_nord, 8, 4, 1, "continentale", f"champion d'amerique du nord {debut + i}")
            competition(equipes_sud, 0, 2, 2, "continentale", f"champion d'amerique du sud {debut + i}")
        elif i % 4 == 1:
            competition(equipes_oceanie, 0, 2, 1, "continentale", f"champion d'oceanie {debut + i}")
            competition(equipes_europe, 16, 8, 1, "continentale", f"champion d'europe {debut + i}")
        elif i % 4 == 2:
            competition(equipes_afrique, 16, 8, 1, "continentale", f"champion d'afrique {debut + i}")
            competition(equipes_nord, 8, 4, 1, "continentale", f"champion d'amerique du nord {debut + i}")
            competition(equipes_sud, 0, 2, 2, "continentale", f"champion d'amerique du sud {debut + i}")
            competition(equipes_asie, 8, 4, 1, "continentale", f"champion d'asie {debut + i}")
        elif i % 4 == 3:
            competition(equipes, 24, 16, 1, "mondiale", f"champion du monde {debut + i}")
        if i % 12 == 0:
            for equipe in equipes: 
                equipe.update_joueurs()
                equipe.set_equipe()
        i += 1

        equipes = sorted(equipes, key=lambda x: x.get_elo(), reverse=True)
        for equipe in equipes:
            update_equipes_database(equipe.nom, equipe.elo, equipe.palmarès["mondiale"], equipe.palmarès["continentale"], equipe.statistiques["victoires"], equipe.statistiques["nuls"], equipe.statistiques["defaites"])

    for equipe in equipes: equipe.update_joueurs()
    end = time.time()
    print(f"Temps : {end-start}")   


Cap-Vert 0 - 0 Tchad
Sénégal 2 - 0 Zambie
Cap-Vert 3 - 0 Zambie
Tchad 2 - 1 Sénégal
Cap-Vert 2 - 1 Sénégal
Zambie 1 - 2 Tchad
Cap-Vert (1058.606) | Points : 7 | Buts Marqués : 5 | Buts Encaissés : 1 | Différences : 4
Tchad (1056.242) | Points : 7 | Buts Marqués : 4 | Buts Encaissés : 2 | Différences : 2
Sénégal (976.845) | Points : 3 | Buts Marqués : 4 | Buts Encaissés : 4 | Différences : 0
Zambie (878.416) | Points : 0 | Buts Marqués : 1 | Buts Encaissés : 7 | Différences : -6
Djibouti 4 - 1 Sierra Leone
Maurice 2 - 1 Algérie
Djibouti 6 - 0 Algérie
Sierra Leone 0 - 0 Maurice
Djibouti 3 - 3 Maurice
Algérie 0 - 5 Sierra Leone
Djibouti (1060.813) | Points : 7 | Buts Marqués : 13 | Buts Encaissés : 4 | Différences : 9
Maurice (1028.475) | Points : 5 | Buts Marqués : 5 | Buts Encaissés : 4 | Différences : 1
Sierra Leone (1013.203) | Points : 4 | Buts Marqués : 6 | Buts Encaissés : 4 | Différences : 2
Algérie (879.581) | Points : 0 | Buts Marqués : 1 | Buts Encaissés : 13 | Différences : -1

KeyboardInterrupt: 

In [ ]:
liste = ["equipe "+str(i) for i in range(16)]
liste

['equipe 0',
 'equipe 1',
 'equipe 2',
 'equipe 3',
 'equipe 4',
 'equipe 5',
 'equipe 6',
 'equipe 7',
 'equipe 8',
 'equipe 9',
 'equipe 10',
 'equipe 11',
 'equipe 12',
 'equipe 13',
 'equipe 14',
 'equipe 15']

In [ ]:
def creer_groupe(n, liste):
    lst = [[] for i in range(n)]
    for i in range(len(liste)):
        lst[i % n].append(liste[i])
    return lst

groupe = creer_groupe(2,liste)
groupe

[['equipe 0',
  'equipe 2',
  'equipe 4',
  'equipe 6',
  'equipe 8',
  'equipe 10',
  'equipe 12',
  'equipe 14'],
 ['equipe 1',
  'equipe 3',
  'equipe 5',
  'equipe 7',
  'equipe 9',
  'equipe 11',
  'equipe 13',
  'equipe 15']]

In [ ]:
def generate_ligue(liste):
    matchs = []
    if len(liste) % 2 == 1: liste.append(None)
    for j in range(len(liste) - 1):
        for i in range(len(liste) // 2):
            matchs.append((liste[i], liste[len(liste) - 1 - i]))
        liste.insert(1, liste.pop())
    matchs = [match for match in matchs if not None in match]
    if None in liste: liste.remove(None)
    return matchs

ligue1 = generate_ligue(groupe[0])
ligue2 = generate_ligue(groupe[1])
ligue1

[('equipe 0', 'equipe 14'),
 ('equipe 2', 'equipe 12'),
 ('equipe 4', 'equipe 10'),
 ('equipe 6', 'equipe 8'),
 ('equipe 0', 'equipe 12'),
 ('equipe 14', 'equipe 10'),
 ('equipe 2', 'equipe 8'),
 ('equipe 4', 'equipe 6'),
 ('equipe 0', 'equipe 10'),
 ('equipe 12', 'equipe 8'),
 ('equipe 14', 'equipe 6'),
 ('equipe 2', 'equipe 4'),
 ('equipe 0', 'equipe 8'),
 ('equipe 10', 'equipe 6'),
 ('equipe 12', 'equipe 4'),
 ('equipe 14', 'equipe 2'),
 ('equipe 0', 'equipe 6'),
 ('equipe 8', 'equipe 4'),
 ('equipe 10', 'equipe 2'),
 ('equipe 12', 'equipe 14'),
 ('equipe 0', 'equipe 4'),
 ('equipe 6', 'equipe 2'),
 ('equipe 8', 'equipe 14'),
 ('equipe 10', 'equipe 12'),
 ('equipe 0', 'equipe 2'),
 ('equipe 4', 'equipe 14'),
 ('equipe 6', 'equipe 12'),
 ('equipe 8', 'equipe 10')]

In [ ]:
def match(e1, e2):
    return {e1: {"but": random.randint(0,4)}, e2: {"but": random.randint(0,4)}}

In [ ]:
def match_ligue(equipes):
    ligue = generate_ligue(equipes)
    classement = {equipe: {"points": 0, "buts marqués": 0, "buts encaissés": 0} for equipe in equipes}
    current = classement
    dic = dict()
    for matchs in ligue:
        data = match(matchs[0], matchs[1])
        add_match(matchs[0], matchs[1], data[matchs[0]]["but"], data[matchs[1]]["but"])
        classement[matchs[0]]["buts marqués"] += data[matchs[0]]["but"]
        classement[matchs[1]]["buts marqués"] += data[matchs[1]]["but"]
        classement[matchs[0]]["buts encaissés"] += data[matchs[1]]["but"]
        classement[matchs[1]]["buts encaissés"] += data[matchs[0]]["but"]
        if data[matchs[0]]["but"] == data[matchs[1]]["but"]:
            classement[matchs[0]]["points"] += 1
            classement[matchs[1]]["points"] += 1
        elif data[matchs[0]]["but"] < data[matchs[1]]["but"]:
            classement[matchs[1]]["points"] += 3
        elif data[matchs[0]]["but"] > data[matchs[1]]["but"]:
            classement[matchs[0]]["points"] += 3
        current = sorted(sorted(sorted(classement.items(), key=lambda x: x[1]['buts marqués'], reverse=True),
                                key=lambda x: x[1]['buts marqués'] - x[1]['buts encaissés'], reverse=True),
                         key=lambda x: x[1]['points'], reverse=True)
    for i in range(len(current)):
        dic[i]= current[i]
    return current

ligue1 = match_ligue(groupe[0])
ligue1

[('equipe 14', {'points': 14, 'buts marqués': 20, 'buts encaissés': 15}),
 ('equipe 0', {'points': 13, 'buts marqués': 14, 'buts encaissés': 12}),
 ('equipe 6', {'points': 11, 'buts marqués': 15, 'buts encaissés': 12}),
 ('equipe 4', {'points': 10, 'buts marqués': 18, 'buts encaissés': 15}),
 ('equipe 2', {'points': 10, 'buts marqués': 14, 'buts encaissés': 16}),
 ('equipe 12', {'points': 8, 'buts marqués': 12, 'buts encaissés': 13}),
 ('equipe 8', {'points': 7, 'buts marqués': 15, 'buts encaissés': 18}),
 ('equipe 10', {'points': 4, 'buts marqués': 11, 'buts encaissés': 18})]

In [ ]:
def qualifiés(liste, n):
    lst1 = list()
    for i in range(n):
        lst1.append(liste[i])
    return lst1, liste[n]

ligue2 = match_ligue(groupe[1])
ligue2
promu, repeche = qualifiés(ligue1, 2)
promu

[('equipe 14', {'points': 14, 'buts marqués': 20, 'buts encaissés': 15}),
 ('equipe 0', {'points': 13, 'buts marqués': 14, 'buts encaissés': 12})]

In [ ]:
def relegation(liste, n):
    lst1 = list()
    for i in range(n):
        lst1.append(liste[-(i+1)])
    return lst1, liste[-(n+1)]

rele, barrage = relegation(ligue1, 2)
rele

[('equipe 10', {'points': 4, 'buts marqués': 11, 'buts encaissés': 18}),
 ('equipe 8', {'points': 7, 'buts marqués': 15, 'buts encaissés': 18})]

In [ ]:
print(f"{promu} sont promu. {rele} sont relegé. {repeche} et {barrage} vont jouer les barrages")

[('equipe 14', {'points': 14, 'buts marqués': 20, 'buts encaissés': 15}), ('equipe 0', {'points': 13, 'buts marqués': 14, 'buts encaissés': 12})] sont promu. [('equipe 10', {'points': 4, 'buts marqués': 11, 'buts encaissés': 18}), ('equipe 8', {'points': 7, 'buts marqués': 15, 'buts encaissés': 18})] sont relegé. ('equipe 6', {'points': 11, 'buts marqués': 15, 'buts encaissés': 12}) et ('equipe 12', {'points': 8, 'buts marqués': 12, 'buts encaissés': 13}) vont jouer les barrages


In [ ]:
for p in promu:
    print(p[0])

equipe 14
equipe 0


In [ ]:
liste = [1,2,3,4,5,6,7,8,9,0]
def moyenne(liste):
    somme = 0
    i = 0
    for nombre in liste:
        somme += nombre
        i += 1
    return somme / i
    #return somme / len(liste)
print(moyenne(liste))

4.5


In [55]:
def convert_binaire(n):
    binaire = ""
    while n > 0:
        reste = n % 2
        binaire = str(reste) + binaire
        n = n // 2
    return binaire

print(convert_binaire(1000))

1111101000


In [2]:
poids = 80
taille = 1.85
age = 20
kcal = 1.083 * poids + 0.48 * taille + 0.50 * age - 0.13 
print(kcal)

97.39800000000001


In [4]:
a = 1
b = 2
c = 1
print(3*a+2*b+5*c)

12


In [18]:
matricea = [[1, 2, 3],
            [4, 5, 6],
            [7, 8, 9]]
matriceb = [[1, 2, 3],
            [4, 5, 6],
            [7, 8, 9]]
for colonne in matricea:
    for c in colonne:
        print(c)
"""for colonne in matricea:
    som = 0
    for k in range(3):
        for i in range(len(matriceb)):
            som += colonne[i]*matriceb[i][k]
            print(colonne[i])
    print(som)
    k += 1"""

1
2
3
4
5
6
7
8
9


'for colonne in matricea:\n    som = 0\n    for k in range(3):\n        for i in range(len(matriceb)):\n            som += colonne[i]*matriceb[i][k]\n            print(colonne[i])\n    print(som)\n    k += 1'